# Come si fa una blockchain in Python

Una *Blockchain* è un libro mastro digitale in cui le transazioni sono registrate in ordine cronologico e pubblico dotata della caratteristica che una eventuale modifica di una transazione nella catena dei blocchi rende immediatamente invalida ogni successivo blocco. 

![Blockchain](https://cdn-images-1.medium.com/max/2000/1*YrmY7iWMK6xN7vGSPrBYlA.png)



## Creazione dell'oggetto "Block"

Contiene un indice, il tempo di creazione, i dati che porta (payload) e l'hash_del blocco precedente

In [216]:
import hashlib as hasher

class Block:
  def __init__(self, index, timestamp, data, previous_hash):
    self.index = index
    self.timestamp = timestamp
    self.data = data
    self.previous_hash = previous_hash
    self.hash = self.hash_block()
  
  def hash_block(self):
    sha = hasher.sha256()
    sha.update((str(self.index) + 
               str(self.timestamp) + 
               str(self.data) + 
               str(self.previous_hash)).encode())
    return sha.hexdigest()

## Creazione del primo blocco (Genesi)

In [217]:
import datetime as date

def create_genesis_block():
  # Manually construct a block with
  # index zero and arbitrary previous hash
  return Block(0, date.datetime.now(), "Genesis Block", "0")

# Aggiunta di un nuovo blocco al precedente

In [218]:
def next_block(last_block):
  this_index = last_block.index + 1
  this_timestamp = date.datetime.now()
  this_data = "Hey! I'm block " + str(this_index)
  this_hash = last_block.hash
  return Block(this_index, this_timestamp, this_data, this_hash)

# Creazione della Blockchain

In [221]:
# Create the blockchain and add the genesis block
blockchain = [create_genesis_block()]
previous_block = blockchain[0]

# How many blocks should we add to the chain
# after the genesis block
num_of_blocks_to_add = 20

# Add blocks to the chain
for i in range(0, num_of_blocks_to_add):
  block_to_add = next_block(previous_block)
  blockchain.append(block_to_add)
  previous_block = block_to_add
  # Tell everyone about it!
  print ("Block #{} has been added to the blockchain!".format(block_to_add.index))
  print (" Data:  {}".format(block_to_add.data) )
  print (" Hash:  {}\n  Prev: {}\n".format(block_to_add.hash, block_to_add.previous_hash) )

Block #1 has been added to the blockchain!
 Data:  Hey! I'm block 1
 Hash:  71ea5a9b36095617149b54a2abdba47454dc75def2291c9259299c244ecc0619
  Prev: 66bd01d4dc3a4ced13948c321f3a27617f51cb2e83ea7f53c8826dc247f8214b

Block #2 has been added to the blockchain!
 Data:  Hey! I'm block 2
 Hash:  c642b55fcc16ecde27c6c3c627ac3c8346bf3e709f70d9f86c66f12e1ffe8f3c
  Prev: 71ea5a9b36095617149b54a2abdba47454dc75def2291c9259299c244ecc0619

Block #3 has been added to the blockchain!
 Data:  Hey! I'm block 3
 Hash:  be817cc0a9c538f75389914ed08a70620bd696634409dc1d7640d58fedc2f425
  Prev: c642b55fcc16ecde27c6c3c627ac3c8346bf3e709f70d9f86c66f12e1ffe8f3c

Block #4 has been added to the blockchain!
 Data:  Hey! I'm block 4
 Hash:  de8ba49f354b000e9bc45d8b2b5d9fe4135cd9231a45c859b0b569cafcd6d56d
  Prev: be817cc0a9c538f75389914ed08a70620bd696634409dc1d7640d58fedc2f425

Block #5 has been added to the blockchain!
 Data:  Hey! I'm block 5
 Hash:  e0b5ffe5aea01f1adcb10e84aaebf532b0d05778b3b8a587d01512a6a6101e9c

# E adesso... una vera blockchain

L'implementazione di questa è focalizzata  esclusivamente nella funzione del registro hash. Non include alcuna funzionalità avanzata come un libro mastro distribuito o un protocollo di consenso tramite proof-of-work. L'idea della della "transazione" è astratta a un concetto più generale di "messaggio" che può contenere qualsiasi tipo di dati.

L'obiettivo del programma è quello di spiegare e chiarire in che modo una blockchain è strutturata. Non è costruito con l'intenzione di replicare una blockchain avanzata come Bitcoin o Ethereum.

È composta da 3 classi. La classe ``Message()``, la classe ``Block()`` e ``Chain()``.

Un messaggio è il contenitore dati di base. È "sigillato" quando viene aggiunto a un blocco e ha 2 hash che lo identificano: l'hash del payload (contenuto) e l'hash del blocco. Ogni messaggio è collegato al messaggio precedente tramite puntatori hash (l'attributo prev_hash). Il metodo di convalida del messaggio garantirà l'integrità di ogni messaggio, ma _non controllerà se i puntatori di hash sono corretti_. Questo è lasciato al metodo di validazione nella classe ``Block()``.

Un blocco può contenere 1,...,n messaggi collegati in sequenza uno dopo l'altro. Quando un blocco viene aggiunto alla catena, viene a sua volta sigillato e convalidato per garantire che i messaggi siano ordinati correttamente e che i puntatori di hash corrispondano. Una volta che il blocco è stato sigillato e sottoposto a hash, viene convalidato controllando l'atteso rispetto all'effettivo.

Una catena può contenere 1,...,m blocchi collegati in sequenza uno dopo l'altro. L'integrità della catena può essere validata in qualsiasi momento chiamando il metodo di convalida, che chiamerà il metodo di convalida di ciascun blocco e genererà un'eccezione InvalidBlockchain.

In [231]:
import datetime
import hashlib
import time

class Message:
	def __init__(self, data):
		self.hash = None
		self.prev_hash = None
		self.timestamp = time.time()
		self.size = len(data.encode('utf-8'))   # length in bytes
		self.data = data
		self.payload_hash = self._hash_payload()

	def _hash_payload(self):
		return hashlib.sha256(bytearray(str(self.timestamp) + str(self.data), "utf-8")).hexdigest()

	def _hash_message(self):
		return hashlib.sha256(bytearray(str(self.prev_hash) + self.payload_hash, "utf-8")).hexdigest()

	def link(self, message):
		""" Link the message to the previous one via hashes."""
		self.prev_hash = message.hash

	def seal(self):
		""" Get the message hash. """
		self.hash = self._hash_message()

	def validate(self):
		""" Check whether the message is valid or not. """
		if self.payload_hash != self._hash_payload():
			raise InvalidMessage("Invalid payload hash in message: " + str(self))
		if self.hash != self._hash_message():
			raise InvalidMessage("Invalid message hash in message: " + str(self))

	def __repr__(self):
		return 'Message<hash: {}, prev_hash: {}, data: {}>'.format(
			self.hash, self.prev_hash, self.data[:20]
		)

# Il Block 

In [269]:
class Block:
	def __init__(self, *args):
		self.messages = []
		self.timestamp = None
		self.prev_hash = None
		self.hash = None
		if args:
			for arg in args:
				self.add_message(arg)

	def _hash_block(self):
		return hashlib.sha256(bytearray(str(self.prev_hash) + str(self.timestamp) + self.messages[-1].hash, "utf-8")).hexdigest()

	def add_message(self, message):
		if len(self.messages) > 0:
			message.link(self.messages[-1])
		message.seal()
		message.validate()
		self.messages.append(message)
 
	def link(self, block):
		""" The block hash only incorporates the head message hash
			which then transitively includes all prior hashes.
		"""
		self.prev_hash = block.hash
        
	def seal(self):
		self.timestamp = time.time()
		self.hash = self._hash_block()

	def validate(self):
		""" Validates each message hash, then chain integrity, then the block hash.
			Calls each message's validate() method.
			If a message fails validation, this method captures the exception and 
			throws InvalidBlock since an invalid message invalidates the whole block.
		"""
		for i, msg in enumerate(self.messages):
			try:
				msg.validate()
				if i > 0 and msg.prev_hash != self.messages[i-1].hash:
					raise InvalidBlock("Invalid block: Message #{} has invalid message link in block: {}".format(i, str(self)))
				print("  Message {} is valid".format(msg.data))
			except InvalidMessage as ex:
				raise InvalidBlock("Invalid block: Message #{} failed validation: {}. In block: {}".format(
					i, str(ex), str(self))
				)

	def __repr__(self):
		return 'Block<hash: {}, prev_hash: {}, messages: {}, time: {}>'.format(
			self.hash, self.prev_hash, len(self.messages), self.timestamp
		)

# Blockchain

In [270]:
class SimpleChain:
	def __init__(self):
		self.chain = []

	def add_block(self, block):
		""" Add a block if valid."""
		if len(self.chain) > 0:
			block.prev_hash = self.chain[-1].hash
		block.seal()
		block.validate()
		self.chain.append(block)

	def validate(self):
		""" Validates each block, in order.
			An invalid block invalidates the chain.
		"""
		for i, block in enumerate(self.chain):
			try:
				block.validate()
				print("Block {} is valid".format(i))
			except InvalidBlock as exc:
				raise InvalidBlockchain("Invalid blockchain at block number {} caused by: {}".format(i, str(exc)))
		return True

	def __repr__(self):
		return 'SimpleChain<blocks: {}>'.format(len(self.chain))



In [271]:
class InvalidMessage(Exception):
	def __init__(self,*args,**kwargs):
		Exception.__init__(self,*args,**kwargs)

class InvalidBlock(Exception):
	def __init__(self,*args,**kwargs):
		Exception.__init__(self,*args,**kwargs)

class InvalidBlockchain(Exception):
	def __init__(self,*args,**kwargs):
		Exception.__init__(self,*args,**kwargs)


In [272]:
import random
chain = SimpleChain()
block = Block()

N = 20
for j in range(N):
    for i in range(random.randint(1,5)):
        print(("Hi, I'm message {}-{}".format(j,i)))
        block.add_message(Message("Hi, I'm message {}-{}".format(j,i)))
    chain.add_block(block)
    print(block)
    block = Block()
    

Hi, I'm message 0-0
Hi, I'm message 0-1
Hi, I'm message 0-2
Hi, I'm message 0-3
  Message Hi, I'm message 0-0 is valid
  Message Hi, I'm message 0-1 is valid
  Message Hi, I'm message 0-2 is valid
  Message Hi, I'm message 0-3 is valid
Block<hash: 1d3ad6470439196add28116c3fd801620f323340a3ff1bfcbaee130b36406e19, prev_hash: None, messages: 4, time: 1516793975.191057>
Hi, I'm message 1-0
Hi, I'm message 1-1
Hi, I'm message 1-2
Hi, I'm message 1-3
Hi, I'm message 1-4
  Message Hi, I'm message 1-0 is valid
  Message Hi, I'm message 1-1 is valid
  Message Hi, I'm message 1-2 is valid
  Message Hi, I'm message 1-3 is valid
  Message Hi, I'm message 1-4 is valid
Block<hash: 96f11a430f70cb4d43aeab749d38f163fff7e93bd50fcb4dd3c63c18867523e0, prev_hash: 1d3ad6470439196add28116c3fd801620f323340a3ff1bfcbaee130b36406e19, messages: 5, time: 1516793975.1931221>
Hi, I'm message 2-0
  Message Hi, I'm message 2-0 is valid
Block<hash: 272bcec0db7e525e3875dd6b87923545d59e7f57a3d5ffadc06882b988111fbc, prev_

In [273]:
for b in chain.chain:
    print(b)
    print("----------------")

Block<hash: 1d3ad6470439196add28116c3fd801620f323340a3ff1bfcbaee130b36406e19, prev_hash: None, messages: 4, time: 1516793975.191057>
----------------
Block<hash: 96f11a430f70cb4d43aeab749d38f163fff7e93bd50fcb4dd3c63c18867523e0, prev_hash: 1d3ad6470439196add28116c3fd801620f323340a3ff1bfcbaee130b36406e19, messages: 5, time: 1516793975.1931221>
----------------
Block<hash: 272bcec0db7e525e3875dd6b87923545d59e7f57a3d5ffadc06882b988111fbc, prev_hash: 96f11a430f70cb4d43aeab749d38f163fff7e93bd50fcb4dd3c63c18867523e0, messages: 1, time: 1516793975.1940138>
----------------
Block<hash: 7cd29ea6c1bbd28431bdc3c83c0ad3d18b4e2c2b3991ec101ee936dd91c6e793, prev_hash: 272bcec0db7e525e3875dd6b87923545d59e7f57a3d5ffadc06882b988111fbc, messages: 3, time: 1516793975.194991>
----------------
Block<hash: 86be145acb5eb122b244a48c6a76ac610c23eb298604be0a25282bc7e0ecc952, prev_hash: 7cd29ea6c1bbd28431bdc3c83c0ad3d18b4e2c2b3991ec101ee936dd91c6e793, messages: 1, time: 1516793975.19597>
----------------
Block<has

In [274]:
if chain.validate(): print("Integrity validated.")
print(len(chain.chain))

  Message Hi, I'm message 0-0 is valid
  Message Hi, I'm message 0-1 is valid
  Message Hi, I'm message 0-2 is valid
  Message Hi, I'm message 0-3 is valid
Block 0 is valid
  Message Hi, I'm message 1-0 is valid
  Message Hi, I'm message 1-1 is valid
  Message Hi, I'm message 1-2 is valid
  Message Hi, I'm message 1-3 is valid
  Message Hi, I'm message 1-4 is valid
Block 1 is valid
  Message Hi, I'm message 2-0 is valid
Block 2 is valid
  Message Hi, I'm message 3-0 is valid
  Message Hi, I'm message 3-1 is valid
  Message Hi, I'm message 3-2 is valid
Block 3 is valid
  Message Hi, I'm message 4-0 is valid
Block 4 is valid
  Message Hi, I'm message 5-0 is valid
  Message Hi, I'm message 5-1 is valid
  Message Hi, I'm message 5-2 is valid
Block 5 is valid
  Message Hi, I'm message 6-0 is valid
  Message Hi, I'm message 6-1 is valid
  Message Hi, I'm message 6-2 is valid
  Message Hi, I'm message 6-3 is valid
  Message Hi, I'm message 6-4 is valid
Block 6 is valid
  Message Hi, I'm messa

# Falsificazione di un messaggio

In [281]:
print(chain.chain[16])
chain.chain[16].messages[1].data = "Fake News"

Block<hash: 84ee345aa151c97907d3411c212ca39dba81556afc40a099173e8865d48c2771, prev_hash: 39f2a39675106b74da260f3fee5cdbb25db07b28e37496599f00749e27f1601e, messages: 3, time: 1516793975.2110589>


In [278]:
if chain.validate(): print("Integrity validated.")
    

  Message Hi, I'm message 0-0 is valid
  Message Hi, I'm message 0-1 is valid
  Message Hi, I'm message 0-2 is valid
  Message Hi, I'm message 0-3 is valid
Block 0 is valid
  Message Hi, I'm message 1-0 is valid
  Message Hi, I'm message 1-1 is valid
  Message Hi, I'm message 1-2 is valid
  Message Hi, I'm message 1-3 is valid
  Message Hi, I'm message 1-4 is valid
Block 1 is valid
  Message Hi, I'm message 2-0 is valid
Block 2 is valid
  Message Hi, I'm message 3-0 is valid
  Message Hi, I'm message 3-1 is valid
  Message Hi, I'm message 3-2 is valid
Block 3 is valid
  Message Hi, I'm message 4-0 is valid
Block 4 is valid
  Message Hi, I'm message 5-0 is valid
  Message Hi, I'm message 5-1 is valid
  Message Hi, I'm message 5-2 is valid
Block 5 is valid
  Message Hi, I'm message 6-0 is valid
  Message Hi, I'm message 6-1 is valid
  Message Hi, I'm message 6-2 is valid
  Message Hi, I'm message 6-3 is valid
  Message Hi, I'm message 6-4 is valid
Block 6 is valid
  Message Hi, I'm messa

InvalidBlockchain: Invalid blockchain at block number 16 caused by: Invalid block: Message #0 failed validation: Invalid payload hash in message: Message<hash: 56f559b54689336f754695273f597b50797733480267ad9e24b3248bf4f499d0, prev_hash: None, data: Fake News>. In block: Block<hash: 84ee345aa151c97907d3411c212ca39dba81556afc40a099173e8865d48c2771, prev_hash: 39f2a39675106b74da260f3fee5cdbb25db07b28e37496599f00749e27f1601e, messages: 3, time: 1516793975.2110589>